# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop(columns='timestamp')

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [4]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(new_df,reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [5]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [8]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [9]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {
    'n_factors':[30,50,100],
    'reg_all':[0.02,0.03,0.05]
}

g_s_svd = GridSearchCV(SVD,params,n_jobs=-1)
g_s_svd.fit(data)

In [14]:
g_s_svd.best_score

{'rmse': 0.8695679782315701, 'mae': 0.6685224891950281}

In [11]:
# print out optimal parameters for SVD after GridSearch
g_s_svd.best_params

{'rmse': {'n_factors': 30, 'reg_all': 0.05},
 'mae': {'n_factors': 30, 'reg_all': 0.03}}

In [15]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic = cross_validate(knn_basic,data,n_jobs=-1)

In [21]:
# print out the average RMSE score for the test set
print(cv_knn_basic['test_rmse'])
print(np.mean(cv_knn_basic['test_rmse']))

[0.96877058 0.97542085 0.97653209 0.97346945 0.97019354]
0.9728773053018068


In [22]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data,n_jobs=1)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [24]:
# print out the average score for the test set
print(cv_knn_baseline['test_rmse'])
np.mean(cv_knn_baseline['test_rmse'])

[0.87365454 0.87368407 0.88691131 0.88176012 0.87370416]


0.8779428414215691

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [25]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [26]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [27]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [30]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.125890181092769, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [31]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list

In [38]:
# try out the new function here!
user_rating = movie_rater(df_movies,2,'Fantasy')

      movieId                               title                    genres
2823     3771  The Golden Voyage of Sinbad (1973)  Action|Adventure|Fantasy
      movieId                                title  \
7188    72407  Twilight Saga: New Moon, The (2009)   

                                     genres  
7188  Drama|Fantasy|Horror|Romance|Thriller  


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [39]:
df_movies.iloc[2823]

movieId                                  3771
title      The Golden Voyage of Sinbad (1973)
genres               Action|Adventure|Fantasy
Name: 2823, dtype: object

In [43]:
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [46]:
df.query('movieId==72407')

,userId,movieId,rating,timestamp
1208,10,72407,3.0,1455356987
15870,103,72407,2.5,1431955494
17641,111,72407,1.5,1516141802
19045,122,72407,2.0,1461562587
30720,215,72407,2.0,1260908530
41417,280,72407,4.0,1348533858
87448,563,72407,2.5,1441845631
89235,578,72407,3.5,1300990637
97355,605,72407,3.0,1277176141


In [38]:
user_rating

[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [47]:
## add the new ratings to the original ratings DataFrame
user_rating = pd.DataFrame(user_rating)
new_rating_df = pd.concat([new_df,user_rating])
new_data = Dataset.load_from_df(new_rating_df,reader)

In [48]:
# train a model using the new combined DataFrame
svd= SVD(n_factors=50,reg_all=0.05)
svd.fit(new_data.build_full_trainset())

In [57]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append((m_id,svd.predict(1000,m_id)[3]))

In [58]:
list_of_movies

[(1, 3.8075654247039257),
 (3, 3.2574615137883858),
 (6, 3.860502802201365),
 (47, 3.944981606183824),
 (50, 4.082479870052811),
 (70, 3.478297119234237),
 (101, 3.8310654501100636),
 (110, 3.972766336704336),
 (151, 3.6306357435402776),
 (157, 3.3085153668461484),
 (163, 3.6779649456797445),
 (216, 3.545908984204803),
 (223, 3.8435719540737425),
 (231, 3.074370700158427),
 (235, 3.5794975812445533),
 (260, 4.095828450253988),
 (296, 4.096383556249764),
 (316, 3.318334684557511),
 (333, 3.628772292295126),
 (349, 3.538761976997017),
 (356, 4.020243188599961),
 (362, 3.5071343126514507),
 (367, 3.1336278956396253),
 (423, 3.2589614243907525),
 (441, 3.929489759984524),
 (457, 3.94600820429787),
 (480, 3.7258769682157347),
 (500, 3.3745995642282285),
 (527, 4.001552174110357),
 (543, 3.4444650761597897),
 (552, 3.292640853983974),
 (553, 3.8546738612354696),
 (590, 3.7419882974150043),
 (592, 3.3559962931420366),
 (593, 4.077659640805423),
 (596, 3.499978579896111),
 (608, 3.972034874349

In [60]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies)
ranked_movies

[(1, 3.8075654247039257),
 (2, 3.3731625391378803),
 (3, 3.2574615137883858),
 (4, 2.9075065887166796),
 (5, 2.9461219389551396),
 (6, 3.860502802201365),
 (7, 3.063671286599326),
 (8, 3.026513882552033),
 (9, 3.1523109364113613),
 (10, 3.459528659065963),
 (11, 3.645999340743568),
 (12, 2.6622531632311235),
 (13, 3.2836180937046975),
 (14, 3.532601257832609),
 (15, 3.0120092960481624),
 (16, 4.079628091749278),
 (17, 3.690324696318693),
 (18, 3.6153744351327375),
 (19, 2.944701677964833),
 (20, 2.9140703608024965),
 (21, 3.50947397242682),
 (22, 3.3242513144103465),
 (23, 3.0773338489818713),
 (24, 3.0962655924853686),
 (25, 3.397140100584008),
 (26, 3.4031198018797797),
 (27, 3.259136890499093),
 (28, 3.9099582605921195),
 (29, 3.92422446470232),
 (30, 3.287566490982451),
 (31, 3.004923116222573),
 (32, 3.949670032990816),
 (34, 3.502424147966151),
 (36, 3.699688944772202),
 (38, 3.2559335840734236),
 (39, 3.1760826729804106),
 (40, 3.5773067896938184),
 (41, 3.671792053957626),
 (42

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [62]:
user_rating

,userId,movieId,rating
0,1000,3771,3
1,1000,72407,3


In [63]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  0    Toy Story (1995)
Name: title, dtype: object 

Recommendation #  2 :  1    Jumanji (1995)
Name: title, dtype: object 

Recommendation #  3 :  2    Grumpier Old Men (1995)
Name: title, dtype: object 

Recommendation #  4 :  3    Waiting to Exhale (1995)
Name: title, dtype: object 

Recommendation #  5 :  4    Father of the Bride Part II (1995)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.